In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 16, kernel_size=3, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            nn.Dropout(0.05),

            Conv2d(16, 16, kernel_size=3, padding=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            nn.Dropout(0.05),

            Conv2d(16, 32, kernel_size=3, padding=1),
            BatchNorm2d(32),
            ReLU(inplace=True),
            nn.Dropout(0.05),

            MaxPool2d(kernel_size=2),        

            Conv2d(32, 16, kernel_size=1),
            BatchNorm2d(16),
            ReLU(inplace=True),
            nn.Dropout(0.05),

            Conv2d(16, 16, kernel_size=3),
            BatchNorm2d(16),
            ReLU(inplace=True),
            nn.Dropout(0.05),

            Conv2d(16, 16, kernel_size=3),
            BatchNorm2d(16),
            ReLU(inplace=True),
            nn.Dropout(0.05),                

            Conv2d(16, 24, kernel_size=3),
            BatchNorm2d(24),
            ReLU(inplace=True),                

            #MaxPool2d(kernel_size=2),
            

            Conv2d(24, 16, kernel_size=3),
            BatchNorm2d(16),
            ReLU(inplace=True),

            #MaxPool2d(kernel_size=2),
            nn.AdaptiveAvgPool2d((1,1)),
            Conv2d(16, 10, kernel_size=1)

        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [23]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
              ReLU-3           [-1, 16, 28, 28]               0
           Dropout-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           2,320
       BatchNorm2d-6           [-1, 16, 28, 28]              32
              ReLU-7           [-1, 16, 28, 28]               0
           Dropout-8           [-1, 16, 28, 28]               0
            Conv2d-9           [-1, 32, 28, 28]           4,640
      BatchNorm2d-10           [-1, 32, 28, 28]              64
             ReLU-11           [-1, 32, 28, 28]               0
          Dropout-12           [-1, 32, 28, 28]               0
        MaxPool2d-13           [-1, 32, 14, 14]               0
           Conv2d-14           [-1, 16,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    # Return the Test Accuracy
    test_acc = 100. * correct / len(test_loader.dataset)
    return test_acc

In [24]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print('The Epoch number is: ',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test_acc = test(model, device, test_loader)
    # Stop when Test Accuracy exceeds 99.45%
    if (test_acc > 99.45):
      break

  0%|          | 0/469 [00:00<?, ?it/s]

The Epoch number is:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1248, Accuracy: 9676/10000 (96.76%)

The Epoch number is:  2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0225, Accuracy: 9932/10000 (99.32%)

The Epoch number is:  10


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0211, Accuracy: 9938/10000 (99.38%)

The Epoch number is:  11


100%|██████████| 469/469 [00:11<00:00, 42.05it/s]



Test set: Average loss: 0.0192, Accuracy: 9947/10000 (99.47%)

